In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_regression

In [3]:
class MyKNNReg:
    def __init__(self, k=3, metric='euclidean', weight='uniform'):
        self.k = k 
        # Инициализация переменной train_size
        self.train_size = None
        # Инициализация переменных для хранения тренировочных данных
        self.X_train = None
        self.y_train = None
        # Метрика качества
        self.metric = metric
        # Взвещенный KNN
        self.weight = weight
        
       
    def __str__(self):
        # Возвращаем строку в требуемом формате
        return f"MyKNNReg class: k={self.k}"
    
    def fit(self, X: pd.DataFrame, y: pd.Series):
        # Сохраняем X и y внутри объекта класса
        self.X_train = X
        self.y_train = y
        # Записываем размер тренировочной выборки (количество строк, столбцов)
        self.train_size = X.shape  # shape возвращает кортеж (строки, столбцы)
    
      # Вычисление расстояния в зависимости от метрики
    def _compute_distance(self, row1, row2):
        if self.metric == 'euclidean':
            return np.sqrt(np.sum((row1 - row2) ** 2))
        elif self.metric == 'chebyshev':
            return np.max(np.abs(row1 - row2))
        elif self.metric == 'manhattan':
            return np.sum(np.abs(row1 - row2))
        elif self.metric == 'cosine':
            dot_product = np.dot(row1, row2)
            norm1 = np.linalg.norm(row1)
            norm2 = np.linalg.norm(row2)
            return 1 - (dot_product / (norm1 * norm2))
        else:
            raise ValueError(f"Unknown metric: {self.metric}")
        
    # Вычисление весов в зависимости от параметра weight
    def _compute_weights(self, distances):
        if self.weight == 'uniform':
            return np.ones(len(distances))  # Все веса одинаковые
        elif self.weight == 'rank':
            # Вес обратно пропорционален рангу (месту в сортированном списке)
            return 1 / (np.arange(1, len(distances) + 1))
        elif self.weight == 'distance':
            # Вес обратно пропорционален расстоянию (если расстояние равно 0, то вес задаем большим значением)
            distances = np.array(distances)
            with np.errstate(divide='ignore'):  # Игнорируем деление на 0
                weights = 1 / distances
                weights[distances == 0] = np.inf  # Если расстояние равно нулю, вес -> бесконечность
            return weights
        else:
            raise ValueError(f"Unknown weight: {self.weight}")
    
    # Метод для предсказания весов
    def predict(self, X: pd.DataFrame):
        predictions = []
        
        for _, test_row in X.iterrows():
            distances = []
            
            # Вычисляем расстояние до всех точек в тренировочном наборе
            for _, train_row in self.X_train.iterrows():
                dist = self._compute_distance(test_row.values, train_row.values)
                distances.append(dist)
            
            # Находим индексы k ближайших соседей
            nearest_neighbors_indices = np.argsort(distances)[:self.k]
            nearest_neighbors_values = self.y_train.iloc[nearest_neighbors_indices]
            nearest_distances = [distances[i] for i in nearest_neighbors_indices]
            
            # Рассчитываем веса
            weights = self._compute_weights(nearest_distances)
            
            # Взвешиваем таргеты соседей
            if self.weight == 'uniform':
                prediction = np.mean(nearest_neighbors_values)
            else:
                weighted_sum = np.sum(weights * nearest_neighbors_values)
                prediction = weighted_sum / np.sum(weights)
            
            predictions.append(prediction)
        
        return np.array(predictions)

In [5]:
# Тестирование взвешенный KNN
def test_knn_regression(X_train, y_train, X_test, metric, weight, k=3):
    knn_reg = MyKNNReg(k=k, metric=metric, weight=weight)
    knn_reg.fit(pd.DataFrame(X_train), pd.Series(y_train))
    predictions = knn_reg.predict(pd.DataFrame(X_test))
    return predictions

# Генерация данных
X_train, y_train = make_regression(n_samples=100, n_features=3, noise=0.1)
X_test, _ = make_regression(n_samples=10, n_features=3, noise=0.1)

# Тестирование с разными весами
for weight in ['uniform', 'rank', 'distance']:
    print(f"Testing with weight: {weight}")
    predictions = test_knn_regression(X_train, y_train, X_test, metric='euclidean', weight=weight)
    # print(f"Predictions: {predictions}")
    print(f"Sum of predictions: {np.sum(predictions)}\n")

Testing with weight: uniform
Sum of predictions: 51.07423974670592

Testing with weight: rank
Sum of predictions: 44.775165594697405

Testing with weight: distance
Sum of predictions: 37.595022298535845



In [15]:
# Тестирование метрик
def test_knn_regression(X_train, y_train, X_test, metric, k=3):
    knn_reg = MyKNNReg(k=k, metric=metric)
    knn_reg.fit(pd.DataFrame(X_train), pd.Series(y_train))
    predictions = knn_reg.predict(pd.DataFrame(X_test))
    return predictions

# Генерация данных с помощью make_regression
# Набор 1
X_train_1, y_train_1 = make_regression(n_samples=100, n_features=3, noise=0.1)
X_test_1, _ = make_regression(n_samples=10, n_features=3, noise=0.1)

# Набор 2
X_train_2, y_train_2 = make_regression(n_samples=200, n_features=4, noise=0.5)
X_test_2, _ = make_regression(n_samples=10, n_features=4, noise=0.5)

# Набор 3
X_train_3, y_train_3 = make_regression(n_samples=300, n_features=5, noise=1.0)
X_test_3, _ = make_regression(n_samples=10, n_features=5, noise=1.0)

# Метрики для тестирования
metrics = ['euclidean', 'manhattan', 'chebyshev']

# Тестируем с каждой метрикой
for metric in metrics:
    print(f"Testing with metric: {metric}")
    
    # Набор 1
    predictions_1 = test_knn_regression(X_train_1, y_train_1, X_test_1, metric)
    # print(f"Predictions for dataset 1: {predictions_1}")
    print(f"Sum of predictions for dataset 1: {np.sum(predictions_1)}")
    
    # Набор 2
    predictions_2 = test_knn_regression(X_train_2, y_train_2, X_test_2, metric)
    # print(f"Predictions for dataset 2: {predictions_2}")
    print(f"Sum of predictions for dataset 2: {np.sum(predictions_2)}")
    
    # Набор 3
    predictions_3 = test_knn_regression(X_train_3, y_train_3, X_test_3, metric)
    # print(f"Predictions for dataset 3: {predictions_3}")
    print(f"Sum of predictions for dataset 3: {np.sum(predictions_3)}")
    
    print("\n")

Testing with metric: euclidean
Sum of predictions for dataset 1: -359.3011469116941
Sum of predictions for dataset 2: -376.32845160587624
Sum of predictions for dataset 3: 226.22056476070125


Testing with metric: manhattan
Sum of predictions for dataset 1: -323.6837838738266
Sum of predictions for dataset 2: -251.03084394040138
Sum of predictions for dataset 3: 185.02134125656292


Testing with metric: chebyshev
Sum of predictions for dataset 1: -269.7388897853258
Sum of predictions for dataset 2: -376.35892687270876
Sum of predictions for dataset 3: 174.07856472853535




In [4]:
# Тестирование предсказания
def test_knn_regression(X_train, y_train, X_test, k):
    knn_reg = MyKNNReg(k=k)
    knn_reg.fit(pd.DataFrame(X_train), pd.Series(y_train))
    predictions = knn_reg.predict(pd.DataFrame(X_test))
    return np.sum(predictions)

# Генерация данных с помощью make_regression
# Набор 1
X_train_1, y_train_1 = make_regression(n_samples=100, n_features=3, noise=0.1)
X_test_1, _ = make_regression(n_samples=10, n_features=3, noise=0.1)

# Набор 2
X_train_2, y_train_2 = make_regression(n_samples=200, n_features=4, noise=0.5)
X_test_2, _ = make_regression(n_samples=10, n_features=4, noise=0.5)

# Набор 3
X_train_3, y_train_3 = make_regression(n_samples=300, n_features=5, noise=1.0)
X_test_3, _ = make_regression(n_samples=10, n_features=5, noise=1.0)

# Тестируем с k=3
sum_predictions_1 = test_knn_regression(X_train_1, y_train_1, X_test_1, k=3)
sum_predictions_2 = test_knn_regression(X_train_2, y_train_2, X_test_2, k=3)
sum_predictions_3 = test_knn_regression(X_train_3, y_train_3, X_test_3, k=3)

# Вывод суммы предсказаний для каждого набора данных
print(f"Sum of predictions for dataset 1: {sum_predictions_1}")
print(f"Sum of predictions for dataset 2: {sum_predictions_2}")
print(f"Sum of predictions for dataset 3: {sum_predictions_3}")

AttributeError: 'MyKNNReg' object has no attribute 'compute_distance'

In [37]:
# Тестирование предсказания
# Входные данные (параметры для проверки)
input_data = {"k": 3}

# Создание объекта класса
knn_reg = MyKNNReg(k=input_data["k"])

# Пример обучающей выборки
X_train = pd.DataFrame({
    'x': [1, 2, 3],
    'y': [4, 5, 6],
    'z': [7, 8, 9]
})

y_train = pd.Series([10, -20, 30])

# Обучаем модель
knn_reg.fit(X_train, y_train)

# Пример тестовой выборки
X_test = pd.DataFrame({
    'x': [2],
    'y': [4],
    'z': [7]
})

# Делаем предсказание
predictions = knn_reg.predict(X_test)

# Сумма предсказаний
predictions_sum = np.sum(predictions)
print(predictions_sum)

6.666666666666667


In [31]:
# Тестирование обучения
def test_knn_with_datasets():
    # Список параметров для тестов
    test_params = [
        {"n_samples": 50, "n_features": 5, "n_informative": 2, "noise" : 5},
        {"n_samples": 100, "n_features": 10, "n_informative": 5, "noise" : 2},
        {"n_samples": 200, "n_features": 20, "n_informative": 10, "noise" : 3}
    ]

    # Создаем объект KNN
    knn = MyKNNReg()

    # Проходим по каждому набору параметров
    for idx, params in enumerate(test_params):
        # Генерация данных
        X, y = make_regression(n_samples=params['n_samples'],
                                   n_features=params['n_features'],
                                   n_informative=params['n_informative'],
                                   noise=params['noise'],
                                   random_state=42)

        # Преобразование в pandas DataFrame и Series
        X_df = pd.DataFrame(X)
        y_series = pd.Series(y)

        # Обучение модели
        knn.fit(X_df, y_series)

        # Печать результата для каждого набора данных
        print(f"Test {idx + 1}:")
        print(f"Train size: {knn.train_size}")
        print("-" * 40)

test_knn_with_datasets()

Test 1:
Train size: (50, 5)
----------------------------------------
Test 2:
Train size: (100, 10)
----------------------------------------
Test 3:
Train size: (200, 20)
----------------------------------------


In [32]:
# Тестирование класса
knn1 = MyKNNReg()
knn2 = MyKNNReg(5)
knn3 = MyKNNReg(10)

# Проверка
print(knn1)
print(knn2)
print(knn3)

MyKNNReg class: k=3
MyKNNReg class: k=5
MyKNNReg class: k=10
